In [5]:
from google.colab import files
import pandas as pd
import io
from flair.data import Sentence
from flair.models import SequenceTagger
from transformers import LukeTokenizer, LukeForEntitySpanClassification

uploaded = files.upload()


Saving processed_df.csv to processed_df (1).csv


In [6]:
df = pd.read_csv(io.BytesIO(uploaded['processed_df.csv']), index_col=0)
df

,Name,Race
0,Scott,White
1,Kathleen,White
2,Matthew,White
3,Jeffrey,White
4,Mark,White
...,...,...
8475,Mario Tejeda,Hispanic
8476,Mario Tavarez,Hispanic
8477,Mario Taveras,Hispanic
8478,Mario Villatoro,Hispanic


In [121]:
# load tagger
tagger = SequenceTagger.load("flair/ner-english")

def get_flair_result(input_string):
  # make example sentence
  sentence = Sentence(input_string)

  # predict NER tags
  tagger.predict(sentence)
  total_string = ""
  for entity in sentence.get_spans('ner'):
      total_string += str(entity)
      total_string += "\n"
  return total_string


2021-10-11 16:12:32,531 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4


In [ ]:
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")
model = LukeForEntitySpanClassification.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")

In [71]:
def get_num_entities(input_text):
  split_text = input_text.split("\n")
  n_entities = 0
  for i in split_text:
    split_i = i.split(" ")
    n_entities += (len(split_i) - 1)
  return n_entities

def get_true_num_entities(input_text):
  return len((input_text).split(" "))

def all_person_entities(input_text):
  split_text = input_text.split("\n")
  for i in split_text:
    if len(i) > 0 and i[-3:] != "PER":
      return False

  return True

def get_luke_entity(input_text):
  split_text = input_text.split(" ")
  word_start_positions = [0]
  word_end_positions = [len(split_text[0])]
  for i in split_text[1:]:
    word_start_positions.append(word_end_positions[-1] + 1)
    word_end_positions.append(len(i) + word_start_positions[-1])

  entity_spans = []
  for i, start_pos in enumerate(word_start_positions):
    for end_pos in word_end_positions[i:]:
      entity_spans.append((start_pos, end_pos))

  inputs = tokenizer(input_text, entity_spans=entity_spans, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits
  predicted_class_indices = logits.argmax(-1).squeeze().tolist()
  if type(predicted_class_indices) == int:
    predicted_class_indices = [predicted_class_indices]
  total_string = ""
  for span, predicted_class_idx in zip(entity_spans, predicted_class_indices):
    if predicted_class_idx != 0:
      total_string += (input_text[span[0]:span[1]] + " " + str(model.config.id2label[predicted_class_idx]) + "\n")
  return total_string

In [73]:
df["luke"] = df["Name"].apply(lambda x: get_luke_entity(x))
df["num_luke_entities"] = df["luke"].apply(lambda x: get_num_entities(x))
df["true_num_entities"] = df["Name"].apply(lambda x: get_true_num_entities(x))
df["all_person_entities"] = df["luke"].apply(lambda x: all_person_entities(x))
df["flair_results"] = df["Name"].apply(lambda x: get_flair_result(x))

Streaming output truncated to the last 5000 lines.
 Jun Xia
Jun Qin PER
 Jun Qin
Jun Qi PER
 Jun Qi
Jun Zhou PER
 Jun Zhou
Jun Zhu PER
 Jun Zhu
Jun Qian PER
 Jun Qian
Jun Jiang PER
 Jun Jiang
Jun Xin PER
 Jun Xin
Jun Zhen PER
 Jun Zhen
Jun Ouyang PER
 Jun Ouyang
Jun Guo PER
 Jun Guo
Jun Xie PER
 Jun Xie
Jun Xue PER
 Jun Xue
Jun Huo PER
 Jun Huo
Jun Ninh PER
 Jun Ninh
Jun Luo PER
 Jun Luo
Jun Khuu PER
 Jun Khuu
Jun Chuong PER
 Jun Chuong
Jun Zhan PER
 Jun Zhan
Jun Zou PER
 Jun Zou
Jun Jia PER
 Jun Jia
Jun Kieu PER
 Jun Kieu
Jun Huang PER
 Jun Huang
Jun Phuong PER
 Jun Phuong
Jun Nghiem PER
 Jun Nghiem
Jun Myung PER
 Jun Myung
Jun Zeng PER
 Jun Zeng
Jun Tieu PER
 Jun Tieu
Jun Hwang PER
 Jun Hwang
Jun Quach PER
 Jun Quach
Jun Choi PER
 Jun Choi
Jun Truong PER
 Jun Truong
Jun Auyeung PER
 Jun Auyeung
Jun Kwon PER
 Jun Kwon
Jun Li PER
 Jun Li
Jun Radhakrishnan PER
 Jun Radhakrishnan
Jun Phung PER
 Jun Phung
Jun Jin PER
 Jun Jin
Jun Byun PER
 Jun Byun
Jun Liang PER
 Jun Liang
Jun Vuong PER
 

In [75]:
example_sentences = ["I spoke with {}", "{} went to the store"]

In [113]:
# df.loc[(df["all_person_entities"]!= True) | (df["true_num_entities"] != df["num_luke_entities"])]["Race"].value_counts()

Black       150
Hispanic     63
API          58
White        13
Name: Race, dtype: int64

In [ ]:
# !pip install datasets
# from datasets import load_dataset
# dataset = load_dataset(
#    'conll2003')



# Scratch Code: 

In [ ]:
# # load tagger
# tagger = SequenceTagger.load("flair/ner-english")

# # make example sentence
# sentence = Sentence("George Washington went to Washington")

# # predict NER tags
# tagger.predict(sentence)

# # print sentence
# print(sentence)

# # print predicted NER spans
# print('The following NER tags are found:')
# # iterate over entities and print
# for entity in sentence.get_spans('ner'):
#     print(entity)


In [ ]:

# text = "Beyoncé lives in Los Angeles"

# # List all possible entity spans in the text
# word_start_positions = [0, 8, 14, 17, 21]  # character-based start positions of word tokens
# word_end_positions = [7, 13, 16, 20, 28]  # character-based end positions of word tokens
# entity_spans = []
# for i, start_pos in enumerate(word_start_positions):
#   for end_pos in word_end_positions[i:]:
#     entity_spans.append((start_pos, end_pos))

# inputs = tokenizer(text, entity_spans=entity_spans, return_tensors="pt")
# outputs = model(**inputs)
# logits = outputs.logits
# predicted_class_indices = logits.argmax(-1).squeeze().tolist()
# for span, predicted_class_idx in zip(entity_spans, predicted_class_indices):
#   if predicted_class_idx != 0:
#     print(text[span[0]:span[1]], model.config.id2label[predicted_class_idx])
